# 03. Dataset Analysis

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Load the Dataset

In [3]:
df = pd.read_csv("..\data\processed\modeldataset.csv")
df.head()

,instance,VariablesVari.,BinariesBina.,IntegersInte.,ContinuousCont.,ConstraintsCons.,Nonz.Nonz.,param_id,runtime,gap,status,objval
0,30n20b8,18380,18318,62,0,576,109706,0,2.487,0.0,2,302.0
1,30n20b8,18380,18318,62,0,576,109706,1,90.003,0.0,9,302.0
2,30n20b8,18380,18318,62,0,576,109706,2,90.005,0.0,9,302.0
3,30n20b8,18380,18318,62,0,576,109706,3,4.297,0.0,2,302.0
4,30n20b8,18380,18318,62,0,576,109706,4,69.428,0.0,2,302.0


## Overview Analysis

- parameter sets counts --> just to check the completeness of the dataset
- 

#### Parameter sets counts

In [ ]:
param_counts = df["param_id"].value_counts().sort_index()
param_counts.plot(kind="bar", title="Parameter ID Distribution", xlabel="param_id", ylabel="Count")